In [1]:
!pip install pandas polars bs4 openpyxl pyarrow

import pandas as pd
import polars as pl
import bs4
import requests
import re
import os

  Using cached openpyxl-3.1.2-py2.py3-none-any.whl (249 kB)
  Using cached et_xmlfile-1.1.0-py3-none-any.whl (4.7 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.0/24.0 MB 5.9 MB/s eta 0:00:0000:0100:01


/Users/mfelipemota/Projects/olho-no-combustivel/.venv/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
# get all links that contains .csv 
relative_path = './levantamento_semanal_precos/'
url = 'https://www.gov.br/anp/pt-br/assuntos/precos-e-defesa-da-concorrencia/precos/levantamento-de-precos-de-combustiveis-ultimas-semanas-pesquisadas'
response = requests.get(url)
soup = bs4.BeautifulSoup(response.text, 'html.parser')
links = soup.find_all('a', href=re.compile(r'revendas'))
# get only links between href and " target
links = [link['href'] for link in links if 'target' in link.attrs]

links
# # download all files using multiprocessing and urllib
import urllib.request
from multiprocessing.pool import ThreadPool

def download_file(link):
    # from string https://www.gov.br/anp/pt-br/centrais-de-conteudo/dados-abertos/arquivos/pmqc/2023/pmqc_2023_04.csv
    # get year of file 
    year = link.split('/')[-2]
    name_file = link.split('/')[-1]
    if not os.path.exists(f"./levantamento_semanal_precos/"):
        os.makedirs(f"./levantamento_semanal_precos/")
    try:
        urllib.request.urlretrieve(link, f"./levantamento_semanal_precos/{name_file}")
        print(f"Downloaded {name_file}")
    except:
        print(f"Error to download {name_file}")

pool = ThreadPool(8)
pool.map(download_file, links)
pool.close()



Downloaded revendas_lpc_2023-07-09_2023-07-15.xlsx
Downloaded revendas_lpc_2023-05-28_2023-06-03.xlsx
Downloaded revendas_lpc_2023-09-10_2023-09-16.xlsx
Downloaded revendas_lpc_2023-05-07_2023-05-13.xlsx
Downloaded revendas_lpc_2023-08-20_2023-08-26.xlsx
Downloaded revendas_lpc_2023-06-18_2023-06-24.xlsx
Downloaded revendas_lpc_2023-12-24_2023-12-30.xlsx
Downloaded revendas_lpc_2023-07-30_2023-08-05.xlsx
Downloaded revendas_lpc_2023-12-17_2023-12-23.xlsx
Downloaded revendas_lpc_2023-08-13_2023-08-19.xlsx
Downloaded revendas_lpc_2023-06-11_2023-06-17.xlsx
Downloaded revendas_lpc_2023-07-02_2023-07-08.xlsx
Downloaded revendas_lpc_2023-09-03_2023-09-09.xlsx
Downloaded revendas_lpc_2023-05-21_2023-05-27.xlsx
Downloaded revendas-lpc-3004-0605.xlsx
Downloaded revendas_lpc_2023-06-25_2023-07-01.xlsx
Downloaded revendas_lpc_2023-07-23_2023-07-29.xlsx
Downloaded revendas_lpc_2023-06-04_2023-06-10.xlsx
Downloaded revendas_lpc_2023-05-14_2023-05-20.xlsx
Downloaded revendas_lpc_2023-08-06_2023-08-

FileExistsError: [Errno 17] File exists: './levantamento_semanal_precos/'

In [3]:
# read all execels inside folder and concat to polars dataframe

# get all files inside folder
files = os.listdir(relative_path)
# get only files that contains .xlsx
files = [file for file in files if '.xlsx' in file]
# read all files and concat to dataframe

df = pd.DataFrame() 
for file in files:
    try:
        df = pd.concat([df, pd.read_excel(f"{relative_path}{file}", engine='openpyxl')])
    except:
        pass
df


/Users/mfelipemota/Projects/olho-no-combustivel/.venv/lib/python3.9/site-packages/openpyxl/reader/workbook.py:84: UserWarning: File contains an invalid specification for 0. This will be removed
  warn(msg)
/Users/mfelipemota/Projects/olho-no-combustivel/.venv/lib/python3.9/site-packages/openpyxl/reader/workbook.py:102: UserWarning: Defined names for sheet index 0 cannot be located
  warn(f"Defined names for sheet index {idx} cannot be located")
/Users/mfelipemota/Projects/olho-no-combustivel/.venv/lib/python3.9/site-packages/openpyxl/reader/workbook.py:84: UserWarning: File contains an invalid specification for 0. This will be removed
  warn(msg)
/Users/mfelipemota/Projects/olho-no-combustivel/.venv/lib/python3.9/site-packages/openpyxl/reader/workbook.py:102: UserWarning: Defined names for sheet index 0 cannot be located
  warn(f"Defined names for sheet index {idx} cannot be located")
/Users/mfelipemota/Projects/olho-no-combustivel/.venv/lib/python3.9/site-packages/openpyxl/reader/work

,"AGÊNCIA NACIONAL DO PETRÓLEO, GÁS NATURAL E BIOCOMBUSTÍVEIS - ANP",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14
0,SUPERINTENDÊNCIA DE DEFESA DA CONCORRÊNCIA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,SISTEMA DE LEVANTAMENTO DE PREÇOS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,INTERVALO DE TEMPO: SEMANAL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,PERÍODO: 14/05/2023 A 20/05/2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17957,36939426000152,FL COMERCIO DE COMBUSTIVEIS LTDA,NaN,AVENIDA DEUSDEDITH SALGADO,4992,AREA 1-A3,TEIXEIRAS,36033000,JUIZ DE FORA,MINAS GERAIS,VIBRA ENERGIA,GASOLINA COMUM,R$ / litro,5.09,2023-01-13 00:00:00
17958,5729222000325,J. C. S. GUIMARAES LTDA,NaN,RUA RANOLFO DE SOUZA GATO,1106,NaN,MARABAIXO 3,68909891,MACAPA,AMAPA,IPIRANGA,DIESEL S10,R$ / litro,6.76,2023-01-13 00:00:00
17959,5729222000325,J. C. S. GUIMARAES LTDA,NaN,RUA RANOLFO DE SOUZA GATO,1106,NaN,MARABAIXO 3,68909891,MACAPA,AMAPA,IPIRANGA,DIESEL S500,R$ / litro,6.45,2023-01-13 00:00:00
17960,5729222000325,J. C. S. GUIMARAES LTDA,NaN,RUA RANOLFO DE SOUZA GATO,1106,NaN,MARABAIXO 3,68909891,MACAPA,AMAPA,IPIRANGA,GASOLINA ADITIVADA,R$ / litro,4.84,2023-01-13 00:00:00


In [6]:
import pyarrow

df_comb = df.copy()
## filter column Unnamed 7 is int
df_comb = df_comb[df_comb['Unnamed: 7'].apply(lambda x: isinstance(x, int))]
df_comb.columns = [
    'CNPJ',
    'RAZAO_SOCIAL',
    'NOME_FANTASIA',
    'ENDERECO',
    'NUMERO',
    'COMPLEMENTO',
    'BAIRRO',
    'CEP',
    'MUNICIPIO',
    'ESTADO',
    'BANDEIRA',
    'PRODUTO',
    'UNIDADE_PRECO',
    'PRECO',
    'DATA_COLETA',
]

df_comb.columns = [col.lower() for col in df_comb.columns]

#df_comb = df_comb[df_comb['estado'] == 'MATO GROSSO DO SUL']

# convert to polars
df_comb = pl.from_pandas(df_comb.astype(str))
# sort by DATA_COLETA desc in polars
df_comb = df_comb.sort('data_coleta', descending=True)
# covert column preco to double in polars
df_comb = df_comb.with_columns(
    pl.col('preco').cast(pl.Float64),
    pl.col('data_coleta').str.to_datetime("%Y-%m-%d 00:00:00"),
    pl.col('cnpj').str.replace(r'\D', '')
)
df_comb = df_comb.with_columns(
        pl.col('cnpj').str.zfill(14)
)

df_comb.write_json('fuel_price_processed/precos.json', row_oriented=True)

In [8]:
## insert postos to MongoDB with column CnpjMatriz as index, and latitute and longitude as 2dsphere

from pymongo import MongoClient
import json
MONGO_URL = "mongodb+srv://***REMOVED***/?retryWrites=true&w=majority"

client = MongoClient(MONGO_URL)
db = client['pmqc']
collection = db['precos']

with open('fuel_price_processed/precos.json') as f:
    file_data = json.load(f)
    collection.insert_many(file_data)


In [ ]:
# get rows for collection postos, coletas and precos
postos = db['postos']
coletas = db['coletas']
precos = db['precos']

# get row numbers
print(f"Total de postos: {postos.count_documents({})}")
print(f"Total de coletas: {coletas.count_documents({})}")
print(f"Total de precos: {precos.count_documents({})}")


Total de postos: 41625
Total de coletas: 488901
Total de precos: 793709
